In [1]:
import math
import numpy as np
import torch
import torchsde
import gc  # Garbage Collector

import torch
from torch import nn
import time

import os
import torch
import numpy as np
from torch_geometric.datasets import Planetoid, Coauthor
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops, degree
import os
import torch_geometric.transforms as T

from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.nn import GATConv

from torchdiffeq import odeint
from ogb.nodeproppred import PygNodePropPredDataset
from dgl.data import CoauthorCSDataset, AmazonCoBuyComputerDataset, AmazonCoBuyPhotoDataset
from heterophilic import WebKB

In [2]:
from gn_sde import LatentGraphSDE
from bayesian_gcn import BayesianGCNConv


In [3]:
def load_data(dataset_name, root_path='dataset', transform=None):
    create_results_folder(dataset_name)

    if dataset_name.startswith('ogbn'):
        dataset = PygNodePropPredDataset(name=dataset_name, root=root_path, transform=transform)
    elif dataset_name == 'CoauthorCS':
        dataset = CoauthorCSDataset(raw_dir=f"{root_path}/{dataset_name}", transform=transform)
    elif dataset_name == 'Computer':
        dataset = AmazonCoBuyComputerDataset(raw_dir=f"{root_path}/{dataset_name}", transform=transform)
    elif dataset_name == 'Photo':
        dataset = AmazonCoBuyPhotoDataset(raw_dir=f"{root_path}/{dataset_name}", transform=transform)
    elif dataset_name == ['cornell', 'texas', 'wisconsin']:
        dataset = WebKB(root=path, name=ds, transform=T.NormalizeFeatures())
    else:
        path = os.path.join(os.getcwd(), root_path, dataset_name)
        dataset = Planetoid(path, dataset_name, transform=transform)

    return dataset

def set_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def set_seed(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)

def create_results_folder(data_name):
    folder_name = f'results/{data_name.lower()}'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Folder created: {folder_name}")
    else:
        print(f"Folder already exists: {folder_name}")



In [4]:


def preprocess_data(data, dataset_name):
    if (dataset_name == 'Computer') or (dataset_name == 'CoauthorCS') or (dataset_name == 'Photo'):
        g = data
        X = g.ndata['feat']
        Y = g.ndata['label']
        # edge_index = torch.tensor(g.edges(), dtype=torch.long)
        src, dst = g.edges()
        edge_index = torch.stack([src, dst], dim=0)
        # edge_index = to_undirected(edge_index)  # Convert to undirected if needed

        num_nodes = g.num_nodes()
        train_mask, val_mask,  test_mask = create_masks(num_nodes)

        # Wrap the CoauthorCS dataset into a PyG Data object
        data = Data(x=X, y=Y, edge_index=edge_index, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

    
    edge_index = data.edge_index
    # Compute normalization
    deg = degree(edge_index[0], dtype=torch.float)
    norm = torch.pow(deg, -0.5)
    norm[torch.isinf(norm)] = 0
    
    X = data.x
    Y = data.y.squeeze()

    if dataset_name.startswith('ogbn'):
        split_idx = data.get_idx_split()
        train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

        train_mask[split_idx['train']] = True
        val_mask[split_idx['valid']] = True
        test_mask[split_idx['test']] = True
        
    else:
        train_mask = data.train_mask
        val_mask = data.val_mask
        test_mask = data.test_mask

    return X, Y, train_mask, val_mask, test_mask, edge_index, norm

def create_masks(num_nodes, train_percent=0.8, val_percent=0.1):
    if train_percent + val_percent > 1.0:
        raise ValueError("Sum of training and validation percentages cannot exceed 100%.")

    all_indices = torch.randperm(num_nodes)
    train_size = int(num_nodes * train_percent)
    val_size = int(num_nodes * val_percent)

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[all_indices[:train_size]] = True
    val_mask[all_indices[train_size:train_size + val_size]] = True
    test_mask[all_indices[train_size + val_size:]] = True

    return train_mask, val_mask, test_mask




Get Random Seed and Device

In [5]:
device = set_device()
print('device', device)
set_seed(0)

device cuda:0


Load Dataset

In [6]:
# For example, to load the 'Cora' dataset:
data_name = 'Cora'
dataset = load_data(data_name)

# Assuming you have already loaded a dataset into the variable 'dataset'
data = dataset[0].to(device)  # Move data to the specified device
X, Y, train_mask, val_mask, test_mask, edge_index, norm = preprocess_data(data, data_name)
num_feats = X.shape[1]
# batch_size = 64
# num_feats = batch_size
n_classes = len(set(np.array(Y.cpu())))

Folder already exists: results/cora


### Models

In [7]:

class MyGCN(torch.nn.Module):
    def __init__(self, edge_index, in_feats, out_feats, activation=None, dropout=0):
        super(MyGCN, self).__init__()
        self.edge_index = edge_index
        self.dropout = dropout
        self.activation = activation
        self.conv1 = GCNConv(in_feats, out_feats)

    def forward(self, x):
        x = self.conv1(x, self.edge_index)
        if self.activation is not None:
            x = self.activation(x)
        if self.training:  # Apply dropout only during training
            x = F.dropout(x, p=self.dropout, training=self.training)
        return x


### GN-SDE

In [8]:
n_dimension = 64
gcn_in = MyGCN(edge_index=edge_index, in_feats=num_feats, out_feats=n_dimension, activation=F.relu, dropout=0.4).to(device)
gnn_f = nn.Sequential(
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2)
).to(device)
# Complete gcn_out
gcn_out = nn.Sequential(
    nn.Flatten(),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_classes, activation=nn.Softmax(), dropout=0.).to(device)
    ).to(device)

gn_sde = LatentGraphSDE(in_net=gcn_in, drift_net=gnn_f, out_net=gcn_out).to(device)



self.embeddings {'0': 0, '0.0001': 0, '0.0002': 0, '0.0003': 0, '0.0004': 0, '0.0005': 0, '0.0006': 0, '0.0007': 0, '0.0008': 0, '0.0009': 0, '0.001': 0}
using file gn_sde.py
sigma 1.0
rtol 0.01
t1 0.001


In [9]:
def initialize_graph_neural_sde(device=device, n_dimension=n_dimension, num_classes=n_classes):

    gcn_in = MyGCN(edge_index=edge_index, in_feats=num_feats, out_feats=n_dimension, activation=F.relu, dropout=0.4).to(device)

    gnn_f = nn.Sequential(
    MyGCN(edge_index=edge_index, in_feats=n_dimension +1, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2), # The +1 is for the time.
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=None, dropout=0.2)
    ).to(device)


    # Complete gcn_out
    gcn_out = nn.Sequential(
    nn.Flatten(),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=num_classes, activation=nn.Softmax(), dropout=0.).to(device)
    ).to(device)
        


    # Initialize the Graph Neural SDE model
    graph_neural_sde = LatentGraphSDE(drift_net=gnn_f, in_net=gcn_in, out_net=gcn_out, sde_output_dim=n_dimension).to(device)

    return graph_neural_sde

### GN-ODE

In [10]:

class GraphNeuralODE(torch.nn.Module):
    def __init__(self, ode_func, in_net=None, out_net=None, method='rk4', atol=1e-3, rtol=1e-4):
        super(GraphNeuralODE, self).__init__()
        self.in_net = in_net
        self.projection_net = out_net

        self.ode_func = ode_func
        self.method = method
        
        self.atol = atol
        self.rtol = rtol

    def f(self, t, y):
        return self.ode_func(y)

    def forward(self, x):
        x = self.in_net(x).to(device)
        t = torch.tensor([0, 1]).float().to(device)  # Define the time steps
        t = t.type_as(x)
        # t_span = torch.linspace(t[0], t[1], steps=4)  # Adjust the number of steps as needed
        x = odeint(self.f, x.to(device), t.to(device) , method=self.method, atol=self.atol, rtol=self.rtol)
        ode_output_time_1 = x[-1].to(device)
        x = self.projection_net(ode_output_time_1)
        return x


# we wrap all of this Neural ODE initializer
def initialize_graph_neural_ode():

    # Initialize GCN layers
    gcn_in = MyGCN(edge_index=edge_index, in_feats=num_feats, out_feats=n_dimension, activation=F.relu, dropout=0.4).to(device)
    
    # Define and initialize ODE function
    
    gde_func = nn.Sequential(
        MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=nn.Softplus(), dropout=0.2),
        MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_dimension, activation=None, dropout=0.2)
    ).to(device)


    # Output Layer
    gcn_out = nn.Sequential(
    nn.Flatten(),
    MyGCN(edge_index=edge_index, in_feats=n_dimension, out_feats=n_classes, activation=nn.Softmax(), dropout=0.).to(device)
    ).to(device)
        
    # Initialize Neural ODE
    neural_ode = GraphNeuralODE(in_net=gcn_in, ode_func=gde_func, out_net=gcn_out).to(device)

    return neural_ode



### GCN

In [11]:
class MyGCN2(torch.nn.Module):
    def __init__(self, edge_index, in_feats=num_feats, out_feats=n_classes, activation=None, dropout=0):
        super(MyGCN2, self).__init__()
        self.edge_index = edge_index
        self.dropout = dropout
        self.activation = activation
        self.conv1 = GCNConv(in_feats, n_dimension)
        # self.conv2 = GCNConv(n_dimension, n_dimension)
        self.conv4 = GCNConv(n_dimension, out_feats)

    def forward(self, x):
        x = self.conv1(x, self.edge_index)
        if self.activation is not None:
            x = self.activation(x)
        if self.training:
            x = F.dropout(x, p=0.4)
            
        # x = self.conv2(x, self.edge_index)
        # x = self.activation(x)
        # if self.training:
        #     x = F.dropout(x, p=self.dropout)

        x = self.conv4(x, self.edge_index)
        return F.softmax(x, dim=1)
    
    
gcn_model = MyGCN2(edge_index=edge_index, in_feats=num_feats, out_feats=n_classes, activation=F.softplus, dropout=0.2).to(device)
    

In [12]:
def initialize_gcn_model(device=device, num_feats=num_feats, n_dimension=n_dimension, dropout=0.2):
    model = MyGCN2(edge_index=edge_index, in_feats=num_feats, out_feats=n_classes, activation=F.softplus, dropout=dropout).to(device)
    return model

gcn_model = initialize_gcn_model()

In [13]:
gcn_model

MyGCN2(
  (conv1): GCNConv(1433, 64)
  (conv4): GCNConv(64, 7)
)

### Bayesian GCN

In [14]:
class BayesianGCN(torch.nn.Module):
    def __init__(self, edge_index, in_feats=num_feats, out_feats=n_classes, activation=None, dropout=0):
        super(BayesianGCN, self).__init__()
        """
        Create a Bayesian Graph Convolutional Network with two layers.
        
        Parameters:
            g: Graph structure (e.g., from DGL or PyTorch Geometric)
            num_features: Number of input features
            hidden_units: Number of units in the hidden layer
            num_classes: Number of output classes (or units in the output layer)
        """
        self.edge_index = edge_index
        self.dropout = dropout
        self.activation = activation
        self.bayes_conv1 = BayesianGCNConv(in_feats, n_dimension, prior_mean = 0.0, prior_variance = 1.0) 
        # self.bayes_conv2 = BayesianGCNConv(n_dimension, n_dimension, prior_mean = 0.0, prior_variance = 1.0) 
        self.bayes_conv3 = BayesianGCNConv(n_dimension, out_feats, prior_mean = 0.0, prior_variance = 1.0)

    def forward(self, data, return_kl=True):
        x = data
        x, kl1 = self.bayes_conv1(x, self.edge_index, return_kl=return_kl)
        if self.activation is not None:
            x = self.activation(x)
        # x, kl2 = self.bayes_conv2(x, self.edge_index, return_kl=return_kl)
        # if self.activation is not None:
        #     x = self.activation(x)
        x = F.dropout(x, p=self.dropout)
        x, kl3 = self.bayes_conv3(x, self.edge_index, return_kl=return_kl)
        
        if return_kl:
            total_kl = kl1 + kl3
            return F.softmax(x, dim=1), total_kl
        return F.softmax(x, dim=1)    
        

In [15]:
def initialize_bayesian_gcn_model(device=device, num_feats=num_feats, num_classes=n_classes, dropout=0.2):
    model = BayesianGCN(edge_index=edge_index, in_feats=num_feats, out_feats=num_classes, activation=F.softplus, dropout=dropout).to(device)
    return model

bayes_gcn_model = initialize_bayesian_gcn_model()

### Ensemble of GCN

In [16]:
# Step 1: Train individual models
number_of_ensemble = 5
gcn_models = [initialize_gcn_model() for _ in range(number_of_ensemble)]

In [17]:
def mc_sample_stats(model, data, n_samples=100):
    samples = []
    samples_entropy = []
    for _ in range(n_samples):
        output_probability, _ = model(data)
        sample = output_probability
        samples.append(sample)
        sample_entropy = entropy_fn(sample)
        samples_entropy.append(sample_entropy)
    
    samples_tensor = torch.stack(samples) # to keep it consistant we keep the samples mean and var as torch tensors.
    var = torch.var(samples_tensor, dim=0, unbiased=True).sum(dim=1)  # Sum across features for each sample
    mean = torch.mean(samples_tensor, dim=0)
    mean_entropy = torch.mean(samples_tensor, dim=0)
    
    return var, mean, mean_entropy
    
    
def evaluate_stochastic_model(model, data, n_samples=100, entropy_threshold=np.inf, mask_data=test_mask):
    model.eval()
    with torch.no_grad():
        _, samples_mean, samples_entropy = mc_sample_stats(model, X, n_samples=n_samples)
        acc, num_of_predictions = accuracy_with_entropy_threshold(samples_mean, samples_entropy, Y, entropy_threshold=entropy_threshold, mask_data=mask_data)
    return acc, num_of_predictions



def entropy_fn(prediction_probabilities):
    eps = 1e-9
    p_clipped = torch.clamp(prediction_probabilities, eps, 1 - eps)
    entropy = -(p_clipped * torch.log(p_clipped)).sum(dim=1)
    return entropy
        

def accuracy_with_entropy_threshold(prediction_probabilities, entropy, labels, entropy_threshold, mask_data):
    # Only predictions if the entropy is less than the given threshold
    predictions_probabilities = prediction_probabilities[mask_data]
    labels = labels[mask_data]
    entropy = entropy_fn(prediction_probabilities)
    entropy = entropy[mask_data]
    mask = (entropy <= entropy_threshold) # True if the entropy of the prediction is less or equal to the threshold
    predictions_bellow_entropy = predictions_probabilities[mask].to('cpu')
    labels = labels[mask].to('cpu') 
    _, pred = predictions_bellow_entropy.max(dim=1)
    correct = float(pred.eq(labels).sum().item())
    num_of_predictions = labels.size(0)
    if num_of_predictions > 0:
        acc = correct / num_of_predictions
    else:
        acc = None  # to avoid division by zero
    return acc, num_of_predictions


def evaluate_with_entropy_threshold(model, data, entropy_threshold=np.inf, mask_data=test_mask):
    model.eval()
    with torch.no_grad():
        prediction = model(X)
        entropy = entropy_fn(prediction)
        acc, num_of_predictions = accuracy_with_entropy_threshold(prediction, entropy, Y, entropy_threshold=entropy_threshold, mask_data=mask_data)
    return acc, num_of_predictions



def ensemble_inference(models, data, entropy_threshold=np.inf, mask_data=test_mask):
    X, Y, _, test_mask = data
    predictions = []
    # Generate predictions from each model
    for model in models:
        model.eval()
        output = model(X)
        predictions.append(output.data.cpu())

    # Aggregate predictions (e.g., using mean)
    ensemble_prediction = torch.mean(torch.stack(predictions), dim=0).to(device)
    # Calculate uncertainty (e.g., using entropy)
    entropies = [entropy_fn(prediction) for prediction in predictions]
    mean_entropy = torch.mean(torch.stack(entropies), dim=0).to(device)
    # variance_entropy = torch.var(torch.stack(entropies), dim=0)

    acc, num_of_predictions = accuracy_with_entropy_threshold(ensemble_prediction, mean_entropy, Y, entropy_threshold=entropy_threshold, mask_data=mask_data)

    return acc, num_of_predictions


def evaluate_model_type(model, data, model_type, entropy_threshold, num_sample=10, mask_data=test_mask):
    if model_type =='deterministic': 
        return evaluate_with_entropy_threshold(model, data, entropy_threshold=entropy_threshold, mask_data=mask_data)
    elif model_type =='stochastic':
        return evaluate_stochastic_model(model, data, n_samples=num_sample, entropy_threshold=entropy_threshold, mask_data=mask_data)
    elif model_type == 'ensemble':
        return ensemble_inference(model, data, entropy_threshold=entropy_threshold, mask_data=mask_data)
    else:
        raise ValueError(f"Invalid model_type: {model_type}. Supported values are 'stochastic', 'deterministic', or 'ensemble'.")

In [18]:
class LinearScheduler(object):
    def __init__(self, iters, maxval=1.00):
        self._iters = max(1, iters)
        self._val = maxval / self._iters
        self._maxval = maxval

    def step(self):
        self._val = min(self._maxval, self._val + self._maxval / self._iters)

    @property
    def val(self):
        return self._val

def accuracy_fn(y_pred, y_true):

    # Convert raw scores to probabilities using softmax if necessary
    if y_pred.shape[1] > 1:  # Assuming class scores are in the second dimension
        y_pred = torch.softmax(y_pred, dim=1)
        predictions = torch.argmax(y_pred, dim=1)
    else:
        # For binary classification, apply sigmoid and threshold at 0.5
        predictions = torch.sigmoid(y_pred) >= 0.5

    correct = torch.eq(predictions, y_true).sum().item()
    total = y_true.shape[0]
    return correct / total

criterion = torch.nn.CrossEntropyLoss()
data = (X, Y, train_mask, test_mask)  # Assuming these are defined
kl_scheduler = LinearScheduler(iters=200)

In [19]:

def train_model(model, criterion, data, steps, verbose_step, kl_scheduler=kl_scheduler, accuracy_func=accuracy_fn, is_stochastic=False):
    optimizer = torch.optim.Adam(model.parameters())
    best_model = None
    best_val_acc = 0.0
    
    for step in range(steps+1):
        model.train()

        if is_stochastic:
            outputs, kl_div = model(X)
            ce_loss = criterion(outputs[train_mask], Y[train_mask])
            total_loss = ce_loss + (kl_div * kl_scheduler.val if kl_scheduler else 0)
        else:
            outputs = model(X)
            total_loss = criterion(outputs[train_mask], Y[train_mask])

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        with torch.no_grad():
            model.eval()
            if is_stochastic:
                outputs, _ = model(X)
            else:
                outputs = model(X)
                
            train_loss = total_loss.item()
            train_acc = accuracy_func(outputs[train_mask], Y[train_mask])
            test_acc = accuracy_func(outputs[test_mask], Y[test_mask]) 
            val_acc = accuracy_func(outputs[val_mask], Y[val_mask])
            
            # Save the best model based on validation accuracy
            if val_acc > best_val_acc:
                print(f'[{step}], Best Model with a Train Accuracy: {train_acc:.3f}, Val Accuracy: {val_acc:.3f}, Test Accuracy: {test_acc:.3f}')
                best_val_acc = val_acc
                best_model = model

        if step % verbose_step == 0:
            print(f'[{step}], Loss: {train_loss:.3f}, Train Accuracy: {train_acc:.3f}, Val Accuracy: {val_acc:.3f}, Test Accuracy: {test_acc:.3f}')
            

    return best_model



In [20]:

def evaluate_uncertainty_quantification(model_initializer, num_experiments, model_name, entropy_thresholds, data, model_type, is_stochastic):
    all_accuracies = {str(entropy_threshold): [] for entropy_threshold in entropy_thresholds}

    for experiment_number in range(num_experiments):
        print("experiment_number", experiment_number)

        # Ensure the model is on the correct device (GPU if available) for training
        model = model_initializer()
        model.to(device)

        # Train model
        model = train_model(model, criterion, data, steps=2000, verbose_step=100, kl_scheduler=kl_scheduler, accuracy_func=accuracy_fn, is_stochastic=is_stochastic)

        for entropy_threshold in entropy_thresholds:
            print('~' * 10 + f' Model: {model_name}, Entropy Threshold: {entropy_threshold}' + '~' * 10)
            test_accuracy, num_pred = evaluate_model_type(model, data, model_type=model_type, entropy_threshold=entropy_threshold) 
            print(f'Entropy Threshold: {entropy_threshold}', 'Number of Predictions', num_pred, 'Test accuracy:', test_accuracy)
            all_accuracies[str(entropy_threshold)].append(test_accuracy)

        # Move model to CPU to free up GPU memory after evaluation (optional, depends on memory constraints)
        model.to('cpu')

        # Clear cached memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        # Explicitly collect garbage
        gc.collect()

    return all_accuracies


def train_ensemble(steps=2000, verbose_step=100, kl_scheduler=None, accuracy_func=accuracy_fn, is_stochastic=False):    
    gcn_models = [initialize_gcn_model().to(device) for _ in range(number_of_ensemble)]
    for model in gcn_models:
        model = train_model(model, criterion, data, steps, verbose_step, kl_scheduler=kl_scheduler, accuracy_func=accuracy_func, is_stochastic=is_stochastic)
    ensemble_model = gcn_models

    acc, num_pred = ensemble_inference(ensemble_model, data)
    return ensemble_model, acc



def evaluate_uncertainty_quantification_ensemble(num_experiments, model_name, entropy_thresholds, data):
    all_accuracies = {str(entropy_threshold): [] for entropy_threshold in entropy_thresholds}

    for experiment_number in range(num_experiments):
        print("experiment_number", experiment_number)
        # Get data with the training percentages

        ensemble_model, acc = train_ensemble()
        # include model
        for entropy_threshold in entropy_thresholds:
            print('~' * 10 + f' Model: {model_name}, Entropy Threshold: {entropy_threshold}' + '~' * 10)
            test_accuracy, num_pred = ensemble_inference(ensemble_model, data, entropy_threshold=entropy_threshold) 
            print(f'Entropy Threshold: {entropy_threshold}', 'Number of Predictions', num_pred, 'Test accuracy:', test_accuracy)
            # Assuming 'test_accuracy' is the accuracy of the model on the test set
            all_accuracies[str(entropy_threshold)].append(test_accuracy)
            
        
    # Move model to CPU to free up GPU memory after evaluation (optional, depends on memory constraints)
    for model in ensemble_model:
        model.to('cpu')

    # Clear cached memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Explicitly collect garbage
    gc.collect()

    return all_accuracies

In [21]:

import json
def save_results(all_accuracies, model_name, results_path):
    filename = f'{results_path}/accuracies_{model_name}.json'

    # Check if the file already exists
    counter = 1
    while os.path.exists(filename):
        filename = f'{results_path}/accuracies_{model_name}_(%d).json' % counter
        counter += 1

    with open(filename, 'w') as fp:
        json.dump(all_accuracies, fp, indent=4)


### Train Models

In [22]:
num_experiments = 1
entropy_results_path = f'results/{data_name.lower()}' 
entropy_thresholds = [np.inf, 2, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

In [23]:
criterion = torch.nn.CrossEntropyLoss()
data = (X, Y, train_mask, test_mask)  # Assuming these are defined
kl_scheduler = LinearScheduler(iters=200)

### Graph Neural SDE

In [24]:
model_sde_name = 'graph_neural_sde'
graph_neural_sde = initialize_graph_neural_sde

acc_sde = evaluate_uncertainty_quantification(graph_neural_sde, num_experiments, model_sde_name, entropy_thresholds, data, model_type='stochastic', is_stochastic=True)
save_results(acc_sde, model_sde_name, entropy_results_path)


experiment_number 0
self.embeddings {'0': 0, '0.0001': 0, '0.0002': 0, '0.0003': 0, '0.0004': 0, '0.0005': 0, '0.0006': 0, '0.0007': 0, '0.0008': 0, '0.0009': 0, '0.001': 0}
using file gn_sde.py
sigma 1.0
rtol 0.01
t1 0.001


/scratch/rsb63/miniconda3/envs/gn-sde/lib/python3.9/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


[0], Best Model with a Train Accuracy: 0.236, Val Accuracy: 0.162, Test Accuracy: 0.180
[0], Loss: 1.946, Train Accuracy: 0.236, Val Accuracy: 0.162, Test Accuracy: 0.180
[1], Best Model with a Train Accuracy: 0.343, Val Accuracy: 0.206, Test Accuracy: 0.233
[2], Best Model with a Train Accuracy: 0.464, Val Accuracy: 0.290, Test Accuracy: 0.289
[3], Best Model with a Train Accuracy: 0.514, Val Accuracy: 0.354, Test Accuracy: 0.337
[4], Best Model with a Train Accuracy: 0.657, Val Accuracy: 0.386, Test Accuracy: 0.402
[5], Best Model with a Train Accuracy: 0.736, Val Accuracy: 0.456, Test Accuracy: 0.452
[6], Best Model with a Train Accuracy: 0.779, Val Accuracy: 0.504, Test Accuracy: 0.515
[7], Best Model with a Train Accuracy: 0.800, Val Accuracy: 0.534, Test Accuracy: 0.560
[8], Best Model with a Train Accuracy: 0.821, Val Accuracy: 0.560, Test Accuracy: 0.579
[9], Best Model with a Train Accuracy: 0.836, Val Accuracy: 0.598, Test Accuracy: 0.609
[10], Best Model with a Train Accurac

KeyboardInterrupt: 

### Graph Neural ODE

In [ ]:
model_ode_name = 'graph_neural_ode'
graph_neural_ode = initialize_graph_neural_ode

acc_ode = evaluate_uncertainty_quantification(graph_neural_ode, num_experiments, model_ode_name, entropy_thresholds, data, model_type='deterministic', is_stochastic=False)
save_results(acc_ode, model_ode_name, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.358, Val Accuracy: 0.358, Test Accuracy: 0.348
[0], Loss: 2.339, Train Accuracy: 0.358, Val Accuracy: 0.358, Test Accuracy: 0.348
[1], Best Model with a Train Accuracy: 0.376, Val Accuracy: 0.377, Test Accuracy: 0.365
[2], Best Model with a Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365


[100], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[200], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[300], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[400], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[500], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[600], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[700], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[800], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[900], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[1000], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[1100], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[1200], Loss: 2.085, Train Accuracy: 0.376, Val Accuracy: 0.378

### GCN

In [ ]:
initialize_gcn_model()

MyGCN2(
  (conv1): GCNConv(767, 64)
  (conv4): GCNConv(64, 10)
)

In [ ]:
model_gcn_name = 'gcn'
gcn_model = initialize_gcn_model

acc_gcn = evaluate_uncertainty_quantification(gcn_model, num_experiments, model_gcn_name, entropy_thresholds, data, model_type='deterministic', is_stochastic=False)
save_results(acc_gcn, model_gcn_name, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.153, Val Accuracy: 0.172, Test Accuracy: 0.164
[0], Loss: 2.295, Train Accuracy: 0.153, Val Accuracy: 0.172, Test Accuracy: 0.164
[1], Best Model with a Train Accuracy: 0.239, Val Accuracy: 0.261, Test Accuracy: 0.246
[2], Best Model with a Train Accuracy: 0.345, Val Accuracy: 0.359, Test Accuracy: 0.325
[3], Best Model with a Train Accuracy: 0.372, Val Accuracy: 0.380, Test Accuracy: 0.350
[4], Best Model with a Train Accuracy: 0.372, Val Accuracy: 0.381, Test Accuracy: 0.358
[100], Loss: 2.086, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[163], Best Model with a Train Accuracy: 0.378, Val Accuracy: 0.382, Test Accuracy: 0.369
[164], Best Model with a Train Accuracy: 0.379, Val Accuracy: 0.384, Test Accuracy: 0.371
[165], Best Model with a Train Accuracy: 0.388, Val Accuracy: 0.393, Test Accuracy: 0.377
[166], Best Model with a Train Accuracy: 0.392, Val Accuracy: 0.397, Test Accuracy: 0.380
[167], Best 

### Bayesian GCN

In [ ]:
model_bayesian_gcn_name = 'bayesian_gcn'
bayesian_gcn_model = initialize_bayesian_gcn_model


acc_bayesian_gcn = evaluate_uncertainty_quantification(bayesian_gcn_model, num_experiments, model_bayesian_gcn_name, entropy_thresholds, data, model_type='stochastic', is_stochastic=True)
save_results(acc_bayesian_gcn, model_bayesian_gcn_name, entropy_results_path)

experiment_number 0
[0], Best Model with a Train Accuracy: 0.346, Val Accuracy: 0.348, Test Accuracy: 0.327
[0], Loss: 2.311, Train Accuracy: 0.346, Val Accuracy: 0.348, Test Accuracy: 0.327
[1], Best Model with a Train Accuracy: 0.354, Val Accuracy: 0.359, Test Accuracy: 0.343
[2], Best Model with a Train Accuracy: 0.375, Val Accuracy: 0.377, Test Accuracy: 0.365
[4], Best Model with a Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[100], Loss: 2.135, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[200], Loss: 2.134, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[300], Loss: 2.133, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[400], Loss: 2.132, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[500], Loss: 2.130, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[600], Loss: 2.129, Train Accuracy: 0.376, Val Accuracy: 0.378, Test Accuracy: 0.365
[700], Loss: 2.128, Train Accuracy:

### Ensemble of GCN

In [ ]:
model_ensemble = 'ensemble_gcn'

ensemble_acc = evaluate_uncertainty_quantification_ensemble(num_experiments, model_ensemble, entropy_thresholds, data)
save_results(ensemble_acc, model_ensemble, entropy_results_path)

experiment_number 0
[0], Loss: 2.331, Train Accuracy: 0.084, Test Accuracy: 0.081
[100], Loss: 2.086, Train Accuracy: 0.376, Test Accuracy: 0.373
[200], Loss: 2.034, Train Accuracy: 0.440, Test Accuracy: 0.433
[300], Loss: 1.963, Train Accuracy: 0.497, Test Accuracy: 0.491
[400], Loss: 1.950, Train Accuracy: 0.509, Test Accuracy: 0.499
[500], Loss: 1.944, Train Accuracy: 0.512, Test Accuracy: 0.502
[600], Loss: 1.942, Train Accuracy: 0.514, Test Accuracy: 0.503
[700], Loss: 1.940, Train Accuracy: 0.516, Test Accuracy: 0.503
[800], Loss: 1.938, Train Accuracy: 0.517, Test Accuracy: 0.503
[900], Loss: 1.937, Train Accuracy: 0.517, Test Accuracy: 0.505
[1000], Loss: 1.937, Train Accuracy: 0.518, Test Accuracy: 0.505
[1100], Loss: 1.936, Train Accuracy: 0.518, Test Accuracy: 0.506
[1200], Loss: 1.935, Train Accuracy: 0.519, Test Accuracy: 0.506
[1300], Loss: 1.935, Train Accuracy: 0.519, Test Accuracy: 0.507
[1400], Loss: 1.934, Train Accuracy: 0.520, Test Accuracy: 0.506
[1500], Loss: 1.9